In [38]:
import requests
# import smtplib

import matplotlib.pyplot as plt

import os
# import zipfile
import pandas as pd

import seaborn as sns

sns.set()

import plotly.graph_objects as go
#from geopy.geocoders import Nominatim

# Supressing warnings
import warnings

warnings.filterwarnings("ignore")

### Crawler ANP website

In [39]:
### Get files from ANP website

link = [
    "https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/precos-semestrais-ca.zip",
    "https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2022-02.zip",
    "https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2023-01.zip",
    "https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2023-02.zip",
    "https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2024-01.zip",
]

arquivo = [
    "2022_s1.zip",
    "2022_s2.zip",
    "2023_s1.zip",
    "2023_s2.zip",
    "2024_s1.zip",
]

folder_anp = rf"C:\Users\Max\Desktop\Max_Data_science\ANP_Gasolina\ANP_links/"

In [40]:
def download_arquivo(links, arquivo, folder):
    try:
        for i in range(len(links)):
            nome_arquivo_local = os.path.join(folder, arquivo[i])

            response = requests.get(links[i])
            if response.status_code == 200:
                if not os.path.exists(folder):
                    os.makedirs(folder)

                with open(nome_arquivo_local, "wb") as arquivo_local:
                    arquivo_local.write(response.content)
                print(
                    f"Download do arquivo realizado e salvo no diretório {nome_arquivo_local}"
                )
            else:
                print(
                    f"Falha ao baixar o arquivo: {links[i]} com status {response.status_code}"
                )

    except Exception as e:
        print(f"Ocorreu um erro: {e}")


download_arquivo(link, arquivo, folder_anp)

Download do arquivo realizado e salvo no diretório C:\Users\Max\Desktop\Max_Data_science\ANP_Gasolina\ANP_links/2022_s1.zip
Download do arquivo realizado e salvo no diretório C:\Users\Max\Desktop\Max_Data_science\ANP_Gasolina\ANP_links/2022_s2.zip
Download do arquivo realizado e salvo no diretório C:\Users\Max\Desktop\Max_Data_science\ANP_Gasolina\ANP_links/2023_s1.zip
Download do arquivo realizado e salvo no diretório C:\Users\Max\Desktop\Max_Data_science\ANP_Gasolina\ANP_links/2023_s2.zip
Download do arquivo realizado e salvo no diretório C:\Users\Max\Desktop\Max_Data_science\ANP_Gasolina\ANP_links/2024_s1.zip


In [41]:
### Note that some files are zip files


def descompacta_arquivo(folder, folder_unzip):

    for arquivo_zip in os.listdir(folder):
        if arquivo_zip.endswith(".zip") and os.path.isfile(
            os.path.join(folder, arquivo_zip)
        ):
            print(f"Descompactando {arquivo_zip}...")

            try:

                with zipfile.ZipFile(
                    os.path.join(folder, arquivo_zip), "r"
                ) as zip_externo:
                    zip_externo.extractall(path=folder_unzip)
                print(f"{arquivo_zip} descompactado com sucesso em {folder_unzip}")
            except Exception as e:
                print(f"Erro ao descompactar {arquivo_zip}: {e}")


descompacta_arquivo(folder_anp, folder_anp)

Descompactando 2022_s1.zip...
Erro ao descompactar 2022_s1.zip: name 'zipfile' is not defined
Descompactando 2022_s2.zip...
Erro ao descompactar 2022_s2.zip: name 'zipfile' is not defined
Descompactando 2023_s1.zip...
Erro ao descompactar 2023_s1.zip: name 'zipfile' is not defined
Descompactando 2023_s2.zip...
Erro ao descompactar 2023_s2.zip: name 'zipfile' is not defined
Descompactando 2024_s1.zip...
Erro ao descompactar 2024_s1.zip: name 'zipfile' is not defined


### Ingestao de dados

In [42]:
def ingestao_arquivos(
    diretorioarquivo,
    separador,
    cabecalho,
    colunas,
    nomecolunas,
    tipodado,
):

    dfFinal = pd.DataFrame()

    for arquivo in os.listdir(diretorioarquivo):

        path = diretorioarquivo + "\\" + arquivo

        df = pd.read_csv(
            path,
            sep=separador,
            header=cabecalho,
            usecols=colunas,
            names=nomecolunas,
            dtype=tipodado,
        )

        dfFinal = pd.concat([dfFinal, df], ignore_index=True)

    return dfFinal

In [43]:
## Tratar dados
diretorioarquivo = folder_anp
separador = ";"
cabecalho = 0
colunas = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
nomecolunas = [
    "Regiao",
    "UF",
    "Municipio",
    "NomRevenda",
    "CNPJ",
    "NomLogradouro",
    "NrEndereco",
    "NomComplemento",
    "NomBairro",
    "NumCep",
    "Produto",
    "DtColeta",
    "Venda",
    "Compra",
    "UnMedida",
    "Bandeira",
]
tipodado = {
    "Regiao": "str",
    "UF": "str",
    "Municipio": "str",
    "NomRevenda": "str",
    "CNPJ": "str",
    "NrEndereco": "str",
    "NomLogradouro": "str",
    "NomComplemento": "str",
    "NomBairro": "str",
    "NumCep": "str",
    "Produto": "str",
    "DtColeta": "str",
    "Venda": "str",
    "Compra": "str",
    "UnMedida": "str",
    "Bandeira": "str",
}


df_anp = ingestao_arquivos(
    diretorioarquivo,
    separador,
    cabecalho,
    colunas,
    nomecolunas,
    tipodado,
)

### Formatação dos dados

In [44]:
## Formatar variaveis de tempo

df_anp["DtColeta"] = pd.to_datetime(
    df_anp["DtColeta"], format="%d/%m/%Y", errors="coerce"
)

df_anp["Dia"] = df_anp["DtColeta"].dt.day
df_anp["Mes"] = df_anp["DtColeta"].dt.month
df_anp["Ano"] = df_anp["DtColeta"].dt.year

## Formatar Compra para float
df_anp["Venda"] = df_anp["Venda"].str.replace(",", ".").astype(float)

### Identificar postos listados em 2024

In [45]:
df_anp = df_anp[df_anp["Ano"] == 2024]
df_anp

,Regiao,UF,Municipio,NomRevenda,CNPJ,NomLogradouro,NrEndereco,NomComplemento,NomBairro,NumCep,Produto,DtColeta,Venda,Compra,UnMedida,Bandeira,Dia,Mes,Ano
1811502,NE,AL,ARAPIRACA,COMERCIAL DE COMBUSTIVEIS E LUBRIFICANTES VITA...,02.817.655/0001-82,RODOVIA AL-220,1901,NaN,SANTA ESMERALDA,57312-025,GASOLINA,2024-01-01,5.68,NaN,R$ / litro,VIBRA,1,1,2024
1811503,NE,AL,ARAPIRACA,COMERCIAL DE COMBUSTIVEIS E LUBRIFICANTES VITA...,02.817.655/0001-82,RODOVIA AL-220,1901,NaN,SANTA ESMERALDA,57312-025,GASOLINA ADITIVADA,2024-01-01,5.68,NaN,R$ / litro,VIBRA,1,1,2024
1811504,NE,AL,ARAPIRACA,COMERCIAL DE COMBUSTIVEIS E LUBRIFICANTES VITA...,02.817.655/0001-82,RODOVIA AL-220,1901,NaN,SANTA ESMERALDA,57312-025,DIESEL S10,2024-01-01,5.93,NaN,R$ / litro,VIBRA,1,1,2024
1811505,NE,AL,ARAPIRACA,COMERCIAL DE COMBUSTIVEIS E LUBRIFICANTES VITA...,02.817.655/0001-82,RODOVIA AL-220,1901,NaN,SANTA ESMERALDA,57312-025,ETANOL,2024-01-01,3.99,NaN,R$ / litro,VIBRA,1,1,2024
1811506,NE,AL,ARAPIRACA,AUTO POSTO SAO FRANCISCO LTDA,12.706.529/0001-04,RUA SAO FRANCISCO,274,NaN,CENTRO,57300-080,GASOLINA,2024-01-01,5.48,NaN,R$ / litro,VIBRA,1,1,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4577307,S,PR,CAMPO LARGO,AUTO POSTO IMPERADOR DE CAMPO LARGO LTDA,42.182.809/0001-21,RUA XAVIER DA SILVA,1596,NaN,CENTRO,83601-010,ETANOL,2024-06-28,3.99,NaN,R$ / litro,IPIRANGA,28,6,2024
4577308,NE,MA,CAXIAS,MAXXI PETRONORTE COMBUSTIVEIS E DERIVADOS LTDA,30.523.175/0001-44,RUA MANOEL GONCALVES,1458,NaN,CENTRO,65600-110,GASOLINA,2024-06-28,5.79,NaN,R$ / litro,BRANCA,28,6,2024
4577309,NE,MA,CAXIAS,MAXXI PETRONORTE COMBUSTIVEIS E DERIVADOS LTDA,30.523.175/0001-44,RUA MANOEL GONCALVES,1458,NaN,CENTRO,65600-110,GASOLINA ADITIVADA,2024-06-28,5.89,NaN,R$ / litro,BRANCA,28,6,2024
4577310,NE,MA,CAXIAS,MAXXI PETRONORTE COMBUSTIVEIS E DERIVADOS LTDA,30.523.175/0001-44,RUA MANOEL GONCALVES,1458,NaN,CENTRO,65600-110,DIESEL S10,2024-06-28,5.64,NaN,R$ / litro,BRANCA,28,6,2024


### Selecionar Produtos

In [47]:
# df_anp = df_anp[df_anp["Produto"].isin(["GASOLINA", "GASOLINA ADITIVADA"])]
df_anp = df_anp[df_anp["Produto"].isin(["GASOLINA ADITIVADA"])]

### Manter Bandeiras especificas

In [48]:
df_anp["Bandeira"] = df_anp["Bandeira"].replace("VIBRA ENERGIA", "VIBRA")

df_anp["Bandeira"] = df_anp["Bandeira"].replace("RAIZEN MINE", "RAIZEN")

df_anp["Bandeira"] = df_anp["Bandeira"].replace("ALESAT", "ALE")

In [49]:
df_anp = df_anp[
    df_anp["Bandeira"].isin(["BRANCA", "VIBRA", "IPIRANGA", "RAIZEN", "ALE"])
]

### Ajustar CNPJ

In [50]:
# Function to clean and standardize cnpj values
def clean_cnpj(cnpj):
    if pd.isna(cnpj):
        return None  # Handle missing values
    # Convert to string, remove all non-numeric characters
    cnpj = str(cnpj)
    return "".join(filter(str.isdigit, cnpj))


# apply the cleaning function
df_anp["CNPJ"] = df_anp["CNPJ"].apply(clean_cnpj)

### Agrupar por CNPJ-Bandeira

In [51]:
# Group by CNPJ and Bandeira, then aggregate the max values for relevant columns at the max DtColeta
df_unique = (
    df_anp.sort_values(by="DtColeta")
    .groupby(["CNPJ", "Bandeira"])
    .agg(
        {
            "DtColeta": "max",
            "NomLogradouro": "last",  # Get value corresponding to max DtColeta
            "NrEndereco": "last",
            "NomComplemento": "last",
            "NomBairro": "last",
            "NumCep": "last",
            "Regiao": "last",  # You can include other fields similarly
            "UF": "last",
            "Municipio": "last",
            "NomRevenda": "last",
            "Venda": "last",
            "Compra": "last",
            "UnMedida": "last",
        }
    )
    .reset_index()
)

# Display the resulting DataFrame
df_unique[df_unique["CNPJ"] == "00001974000190"]

,CNPJ,Bandeira,DtColeta,NomLogradouro,NrEndereco,NomComplemento,NomBairro,NumCep,Regiao,UF,Municipio,NomRevenda,Venda,Compra,UnMedida


### Remover CNPJ duplicado - Pode estar vinculado a mais de 1 bandeira

In [52]:
duplicated_cnpjs = df_unique["CNPJ"].duplicated(keep=False)
df_2 = df_unique[~duplicated_cnpjs]

### Obter geolocalização de cada cnpj

In [53]:
df_geo = df_2.drop(
    columns=["DtColeta", "NomRevenda", "Venda", "NomComplemento", "Compra", "UnMedida"]
)

### Obter geolocalização de cada cnpj

In [54]:
# Building the address
df_geo["address"] = (
    df_geo["NomLogradouro"]
    + " "
    + df_geo["NrEndereco"]
    + ", "
    + df_geo["NomBairro"]
    + ", "
    + df_geo["Municipio"]
    + ", "
    + df_geo["UF"]
    # + ", Brazil"
)
df_geo

,CNPJ,Bandeira,NomLogradouro,NrEndereco,NomBairro,NumCep,Regiao,UF,Municipio,address
0,00002953000199,IPIRANGA,AVENIDA NOSSA SENHORA DE FATIMA,6 105,VILA AVIACAO,17053-460,SE,SP,BAURU,"AVENIDA NOSSA SENHORA DE FATIMA 6 105, VILA AV..."
1,00005087000190,VIBRA,AVENIDA DE CILLOS,3049,PQ N.MUNDO,13473-291,SE,SP,AMERICANA,"AVENIDA DE CILLOS 3049, PQ N.MUNDO, AMERICANA, SP"
2,00008226000139,VIBRA,AVENIDA BRIG JOSE V F LIMA,1103,JD ANA EMILIA,12070-000,SE,SP,TAUBATE,"AVENIDA BRIG JOSE V F LIMA 1103, JD ANA EMILIA..."
3,00012211000306,RAIZEN,AREA ESPECIAL PLLR,1,GAMA,72460-000,CO,DF,BRASILIA,"AREA ESPECIAL PLLR 1, GAMA, BRASILIA, DF"
4,00017674000107,IPIRANGA,AVENIDA AFONSO SAMPAIO E SOUSA,1428,JD NS DO CARMO,08270-000,SE,SP,SAO PAULO,"AVENIDA AFONSO SAMPAIO E SOUSA 1428, JD NS DO ..."
...,...,...,...,...,...,...,...,...,...,...
7121,97468946000119,IPIRANGA,ESTRADA DOS BANDEIRANTES,4205,JACAREPAGUA,22783-117,SE,RJ,RIO DE JANEIRO,"ESTRADA DOS BANDEIRANTES 4205, JACAREPAGUA, RI..."
7122,97486534000292,VIBRA,QUADRA 204 S AV NS 02,1,CENTRO,77020-486,N,TO,PALMAS,"QUADRA 204 S AV NS 02 1, CENTRO, PALMAS, TO"
7123,97521669000160,IPIRANGA,AVENIDA RIO VERDE,2608,VILA ROSA,74843-660,CO,GO,GOIANIA,"AVENIDA RIO VERDE 2608, VILA ROSA, GOIANIA, GO"
7124,97533147000260,RAIZEN,AVENIDA DOMINGOS DE ALMEIDA,444,AREAL,96085-470,S,RS,PELOTAS,"AVENIDA DOMINGOS DE ALMEIDA 444, AREAL, PELO..."


### Para geolocalização, preciso agrupar por CNPJ - BANDEIRA

Por algum motivo existe erro de digitação na hora da coleta dos dados por parte da ANP

In [55]:
# from geopy.geocoders import Nominatim
# from geopy.extra.rate_limiter import RateLimiter
# import time

# # Initialize the geocoder
# app = Nominatim(user_agent="my_geocoding")

# # Create a RateLimiter to limit to 1 requests per second
# geocode = RateLimiter(app.geocode, min_delay_seconds=1)
# geocode

# first_address = (
#     "RUA VEREADOR ARTHUR MANOEL MARIANO 977, FORQUILHINHAS, SAO JOSE, SC, Brazil"
# )
# # first_address = df_3['address'].iloc[4]

# print("Geocoding address:", first_address)

# try:
#     # Attempt to geocode the address
#     location = app.geocode(first_address)

#     if location:
#         print("Geocoding successful!")
#         print("Raw location data:", location.raw)
#         latitude = location.latitude
#         longitude = location.longitude
#         print(f"Latitude: {latitude}, Longitude: {longitude}")
#     else:
#         print("No location found for:", first_address)

# except Exception as e:
#     print("Error during geocoding:", e)

In [56]:
# df_test = df_2.copy()

# from geopy.extra.rate_limiter import RateLimiter
# import time

# # Initialize the geocoder
# app = Nominatim(user_agent="my_geocoding")


# # Set up the geocoding with a RateLimiter and a longer timeout
# geocode = RateLimiter(
#     app.geocode, min_delay_seconds=1, error_wait_seconds=2, max_retries=3
# )


# # Function to geocode an address
# def geocode_address(address):
#     try:
#         location = app.geocode(address)
#         if location:
#             return location.latitude, location.longitude
#         else:
#             return None, None
#     except Exception as e:
#         print(f"Error during geocoding for {address}: {e}")
#         return None, None


# # Apply geocoding to the first 10 rows of the DataFrame
# df_test["latitude"], df_test["longitude"] = zip(
#     *df_test["address"].apply(geocode_address)
# )

# df_test

In [57]:
df_test[df_test["latitude"].notnull()]

NameError: name 'df_test' is not defined

In [58]:
df_final = df_test[df_test["latitude"].notnull()]

NameError: name 'df_test' is not defined

In [59]:
df_final.to_parquet('df_anp_full.parquet', index = False)

NameError: name 'df_final' is not defined

## Começar a partir daqui

In [60]:
import pandas as pd

import seaborn as sns

sns.set()

In [61]:
df_geo = pd.read_parquet('df_anp_full.parquet')
df_geo

,CNPJ,Bandeira,NomLogradouro,NrEndereco,NomBairro,NumCep,Regiao,UF,Municipio,address,latitude,longitude
0,00002953000270,VIBRA,RUA PADRE FRANCISCO VAN DER MAAS,25750,JARDIM CONTORNO,17047-020,SE,SP,BAURU,"RUA PADRE FRANCISCO VAN DER MAAS 25750, JARDIM...",-22.338475,-49.044851
1,00003188001608,BRANCA,RUA SAO PAULO,246,CENTRO,18130-120,SE,SP,SAO ROQUE,"RUA SAO PAULO 246, CENTRO, SAO ROQUE, SP, Brazil",-23.538073,-49.242850
2,00021475000164,BRANCA,RUA BERNARDINO DE CAMPOS,28307,VILA FALCAO,17050-040,SE,SP,BAURU,"RUA BERNARDINO DE CAMPOS 28307, VILA FALCAO, B...",-22.329419,-49.085422
3,00062382000420,BRANCA,AVENIDA CARDEAL AVELAR BRANDAO VILELA,2505,MATA ESCURA,41110-000,NE,BA,SALVADOR,"AVENIDA CARDEAL AVELAR BRANDAO VILELA 2505, MA...",-12.922264,-38.460519
4,00067750000180,VIBRA,AVENIDA WILSON ALVARENGA,4,BELMONTE,35930-292,SE,MG,JOAO MONLEVADE,"AVENIDA WILSON ALVARENGA 4, BELMONTE, JOAO MON...",-19.813782,-43.155761
...,...,...,...,...,...,...,...,...,...,...,...,...
3061,97005581000862,VIBRA,RUA MARCILIO DIAS,3085,TRES VENDAS,96020-480,S,RS,PELOTAS,"RUA MARCILIO DIAS 3085, TRES VENDAS, PELOTAS...",-31.740442,-52.344231
3062,97005581001249,VIBRA,RUA ALMIRANTE BARROSO,2360,CENTRO,96010-280,S,RS,PELOTAS,"RUA ALMIRANTE BARROSO 2360, CENTRO, PELOTAS, R...",-31.767009,-52.335524
3063,97430284000350,BRANCA,AVENIDA AFRANIO PEIXOTO,SN,PARIPE,40800-570,NE,BA,SALVADOR,"AVENIDA AFRANIO PEIXOTO SN, PARIPE, SALVADOR, ...",-12.851783,-38.474942
3064,97468946000119,IPIRANGA,ESTRADA DOS BANDEIRANTES,4205,JACAREPAGUA,22783-117,SE,RJ,RIO DE JANEIRO,"ESTRADA DOS BANDEIRANTES 4205, JACAREPAGUA, RI...",-22.960239,-43.389699


In [62]:
df_anp = df_2.copy()

In [63]:
df_anp

,CNPJ,Bandeira,DtColeta,NomLogradouro,NrEndereco,NomComplemento,NomBairro,NumCep,Regiao,UF,Municipio,NomRevenda,Venda,Compra,UnMedida
0,00002953000199,IPIRANGA,2024-06-24,AVENIDA NOSSA SENHORA DE FATIMA,6 105,None,VILA AVIACAO,17053-460,SE,SP,BAURU,POSTO JARDIM AMERICA DE BAURU LTDA,5.99,None,R$ / litro
1,00005087000190,VIBRA,2024-06-25,AVENIDA DE CILLOS,3049,None,PQ N.MUNDO,13473-291,SE,SP,AMERICANA,NABAS & CAMARGO LTDA,5.79,None,R$ / litro
2,00008226000139,VIBRA,2024-06-24,AVENIDA BRIG JOSE V F LIMA,1103,None,JD ANA EMILIA,12070-000,SE,SP,TAUBATE,AUTO POSTO MANGUEIRA II LTDA,5.77,None,R$ / litro
3,00012211000306,RAIZEN,2024-06-24,AREA ESPECIAL PLLR,1,SLC,GAMA,72460-000,CO,DF,BRASILIA,DRIVE CAR TRANSPORTES E COMBUSTIVEIS LTDA,5.89,None,R$ / litro
4,00017674000107,IPIRANGA,2024-05-27,AVENIDA AFONSO SAMPAIO E SOUSA,1428,None,JD NS DO CARMO,08270-000,SE,SP,SAO PAULO,AUTO POSTO DOCARMO LTDA,6.39,None,R$ / litro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7121,97468946000119,IPIRANGA,2024-03-25,ESTRADA DOS BANDEIRANTES,4205,None,JACAREPAGUA,22783-117,SE,RJ,RIO DE JANEIRO,POSTO DE SERVICOS VIA 8 LTDA,5.95,None,R$ / litro
7122,97486534000292,VIBRA,2024-03-25,QUADRA 204 S AV NS 02,1,None,CENTRO,77020-486,N,TO,PALMAS,MCM-COM DE DERIVADOS DE PETROLEO LTDA,6.19,None,R$ / litro
7123,97521669000160,IPIRANGA,2024-03-26,AVENIDA RIO VERDE,2608,QD.29 LOTE 15,VILA ROSA,74843-660,CO,GO,GOIANIA,MONACO COMBUSTIVEIS LTDA,5.99,None,R$ / litro
7124,97533147000260,RAIZEN,2024-05-20,AVENIDA DOMINGOS DE ALMEIDA,444,None,AREAL,96085-470,S,RS,PELOTAS,ROSA & FILHO COMERCIO DE COMBUSTIVEIS LTDA - ME,6.19,None,R$ / litro


### Ajustar nome de Vibra Energia para Vibra

In [ ]:
df_anp["Bandeira"] = df_anp["Bandeira"].replace("VIBRA ENERGIA", "VIBRA")

### Ajustes de Formatacao CNPJ

In [ ]:
df_anp["CNPJ"] = df_anp["CNPJ"].str.replace(r"\D", "", regex=True)

## Por algum motivo existe registros duplicados
df_anp.drop_duplicates(subset=["DtColeta", "CNPJ"], keep="first", inplace=True)

### Algumas stats

In [ ]:
df_anp

In [ ]:
unique_count = df_anp["Produto"].nunique()
print(f"Total de produtos: {unique_count}")

# Count occurrences of each unique value
value_counts = df_anp["Produto"].value_counts()
print(value_counts)

unique_count = df_anp["Bandeira"].nunique()
print(f"\nTotal de Bandeirass: {unique_count}")

# Count occurrences of each unique value
value_counts_pct = (df_anp["Bandeira"].value_counts()) / len(df_anp) * 100
print(value_counts_pct)

### 

### Manter apenas Produto = Gasolina Aditivada

In [ ]:
df_anp_adv = df_anp[df_anp["Produto"] == "GASOLINA ADITIVADA"]

### Manter apenas as principais distribuidoras (90% market share)

In [ ]:
df_anp_adv = df_anp_adv[
    df_anp_adv["Bandeira"].isin(["BRANCA", "IPIRANGA", "RAIZEN", "VIBRA"])
]

### Agrupar por cidade e Bandeira

In [ ]:
stats_city = (
    df_anp_adv.groupby(
        [
            "DtColeta",
            "Ano",
            "Mes",
            "Regiao",
            "UF",
            "Municipio",
        ],
        as_index=True,
        sort=True,
    )
    .agg(
        {
            "NomRevenda": ["nunique"],
            "Bandeira": ["nunique"],
            "Venda": ["mean", "std"],
        }
    )
    .round(2)
)

stats_city.columns = stats_city.columns.droplevel(1)
stats_city.rename(
    columns={"NomRevenda": "#_revendas", "Bandeira": "#_Bandeira"}, inplace=True
)
stats_city.reset_index(inplace=True)
# stats_city.sort_values(by=["DtColeta", "Bandeira"], inplace=True)

stats_city.columns = [
    "DtColeta",
    "Ano",
    "Mes",
    "Regiao",
    "UF",
    "Municipio",
    "#_Revendas",
    "#_Bandeira",
    "Preco_venda_medio",
    "Preco_venda_std",
]

stats_city

In [ ]:
stats_city[stats_city["#_Revendas"] > 20].Municipio.value_counts()

In [ ]:
# Supondo que o DataFrame stats_city já esteja carregado com seus dados

# Identifica todas as regiões únicas
regioes = stats_city["Regiao"].unique()

# Configura a quantidade de subplots baseado no número de regiões
fig, axs = plt.subplots(1, len(regioes), figsize=(5 * len(regioes), 5), sharey=True)

# Itera por cada região e cria um scatter plot separado
for i, regiao in enumerate(regioes):
    regiao_data = stats_city[stats_city["Regiao"] == regiao]
    axs[i].scatter(regiao_data["#_Revendas"], regiao_data["Preco_venda_std"])
    axs[i].set_title(f"Região: {regiao}")
    axs[i].set_xlabel("Número de Revendas")
    if i == 0:  # Apenas o primeiro gráfico precisa do rótulo no eixo y
        axs[i].set_ylabel("Preço de Venda (Desvio Padrão)")

# Ajusta o layout para evitar sobreposição
plt.tight_layout()
plt.show()

### Excluir registros em que o std preço está muito grande - Cidades com poucas revendas e algo muito diferente em políticas de preço. 

In [ ]:
stats_city_2 = stats_city[stats_city["Preco_venda_std"] < 1.10]

In [ ]:
# Identifica todas as regiões únicas
regioes = stats_city_2["Regiao"].unique()

# Configura a quantidade de subplots baseado no número de regiões
fig, axs = plt.subplots(1, len(regioes), figsize=(5 * len(regioes), 5), sharey=True)

# Itera por cada região e cria um scatter plot separado
for i, regiao in enumerate(regioes):
    regiao_data = stats_city_2[stats_city_2["Regiao"] == regiao]
    axs[i].scatter(regiao_data["#_Revendas"], regiao_data["Preco_venda_std"])
    axs[i].set_title(f"Região: {regiao}")
    axs[i].set_xlabel("Número de Revendas")
    if i == 0:  # Apenas o primeiro gráfico precisa do rótulo no eixo y
        axs[i].set_ylabel("Preço de Venda (Desvio Padrão)")

# Ajusta o layout para evitar sobreposição
plt.tight_layout()
plt.show()

### Notas:

* O maior grau de dispersão de preços ocorre na região SE - Pode ser por ter mais dados amostrais
* Quanto maior a quantidade de revendas, menor o desviao padrão nos preços - Produto homogeneo e maior concorrência

In [ ]:
stats_city_2

### Reajustes do Governo

Para lidar com a questão dos ajustes de preços que podem não coincidir exatamente com as datas de coleta, você pode usar um intervalo de 7 dias antes e 7 dias depois de cada data de ajuste. Deve-se verificar verifica se a data da coleta está dentro de um intervalo de 14 dias ao redor de cada data de ajuste.


In [ ]:
# Lista de datas de ajustes
ajustes = pd.to_datetime(
    [
        "2022-01-11",
        "2022-03-11",
        "2022-08-16",
        "2022-09-20",
        "2023-04-24",
        "2023-06-16",
        "2024-07-09",
    ]
)

# Para cada data de ajuste, criaremos um intervalo de 7 dias antes e 7 dias depois
intervalos_ajustes = pd.concat(
    [
        pd.Series(
            pd.date_range(
                start - pd.Timedelta(days=7), end=start + pd.Timedelta(days=7)
            )
        )
        for start in ajustes
    ]
)

# Criar a variável dummy que será 1 se a data estiver dentro de qualquer intervalo de ajuste
stats_city_2["ajuste_gov"] = (
    stats_city_2["DtColeta"].isin(intervalos_ajustes).astype(int)
)

In [ ]:
stats_city_2.Municipio.nunique()

In [ ]:
stats_city_2

## Baseline - Modelos Univariados

In [ ]:
df_baseline = stats_city_2[
    [
        "DtColeta",
        "Municipio",
        "Preco_venda_medio",
        "ajuste_gov",
    ]
]

In [ ]:
# Agora você pode tentar filtrar novamente pela cidade

df_city = df_baseline[df_baseline["Municipio"] == "RIO DE JANEIRO"]

In [ ]:
########################
######## CIDADE 1

df_city = df_baseline[df_baseline["Municipio"] == "RIO DE JANEIRO"]

df_city = df_city.sort_values(by="DtColeta")

municipio = df_city["Municipio"].iloc[0]

# Filter periods where 'ajuste_preco' equals 1
adjustment_periods = df_city[df_city["ajuste_gov"] == 1]["DtColeta"]

# Criar uma figura de linha para o preço VENDA
fig = go.Figure()

# Adicionar a linha de preços VENDA ao longo do tempo
fig.add_trace(
    go.Scatter(
        x=df_city["DtColeta"],  # Eixo X como DtColeta, que está em formato datetime
        y=df_city["Preco_venda_medio"],  # Eixo Y como preço de venda
        mode="lines",
        name="Preço VENDA",
    )
)

# Iterate through adjustment periods and add shaded regions
for start_date in adjustment_periods:
    # Assuming each period lasts 14 days as defined by your previous setup
    end_date = start_date + pd.Timedelta(days=13)
    fig.add_vrect(
        x0=start_date,
        x1=end_date,
        fillcolor="red",
        opacity=0.2,
        layer="below",
        line_width=0,
    )

# Definir títulos e ajustes nos eixos
fig.update_layout(
    title=f"Comportamento do Preço de Venda da Gasolina - {municipio}",
    xaxis_title="Data de Coleta",
    yaxis_title="Preço de Venda (R$)",
    xaxis=dict(type="date"),  # Forçar o eixo X a reconhecer como datas
    showlegend=True,
)

# Mostrar o gráfico
fig.show()

########################
######## CIDADE 2

df_city = df_baseline[df_baseline["Municipio"] == "SAO PAULO"]

df_city = df_city.sort_values(by="DtColeta")

municipio = df_city["Municipio"].iloc[0]

# Filter periods where 'ajuste_preco' equals 1
adjustment_periods = df_city[df_city["ajuste_gov"] == 1]["DtColeta"]

# Criar uma figura de linha para o preço VENDA
fig = go.Figure()

# Adicionar a linha de preços VENDA ao longo do tempo
fig.add_trace(
    go.Scatter(
        x=df_city["DtColeta"],  # Eixo X como DtColeta, que está em formato datetime
        y=df_city["Preco_venda_medio"],  # Eixo Y como preço de venda
        mode="lines",
        name="Preço VENDA",
    )
)

# Iterate through adjustment periods and add shaded regions
for start_date in adjustment_periods:
    # Assuming each period lasts 14 days as defined by your previous setup
    end_date = start_date + pd.Timedelta(days=13)
    fig.add_vrect(
        x0=start_date,
        x1=end_date,
        fillcolor="red",
        opacity=0.2,
        layer="below",
        line_width=0,
    )

# Definir títulos e ajustes nos eixos
fig.update_layout(
    title=f"Comportamento do Preço de Venda da Gasolina - {municipio}",
    xaxis_title="Data de Coleta",
    yaxis_title="Preço de Venda (R$)",
    xaxis=dict(type="date"),  # Forçar o eixo X a reconhecer como datas
    showlegend=True,
)

# Mostrar o gráfico
fig.show()

Pelo gráfico se percebe que as coletas da ANP não ocorrem em sincronia de dias. 

In [ ]:
df_baseline

In [ ]:
# Pivot the DataFrame
df_baseline_ts = df_baseline.pivot_table(
    index=["DtColeta"],
    columns="Municipio",
    values="Preco_venda_medio",
    aggfunc="mean",  # or "max", "min", depending on what you want
)

# Reset the index to flatten the DataFrame
df_baseline_ts.reset_index(inplace=True)

df_baseline_ts.set_index("DtColeta", inplace=True)

df_baseline_ts
# df_vibra.sort_values(by=["DtColeta", "Municipio"], ascending=True)

In [ ]:
pct_missing = df_baseline_ts.isnull().sum() / len(df_baseline_ts) * 100

missing_df = pd.DataFrame({"percent_missing": pct_missing})

missing_df.sort_values(by="percent_missing")

### Plot: Evolução temporal dos preços de Gasolinma Aditivada por sku

In [ ]:
# Agora você pode tentar filtrar novamente pelo CNPJ
cnpj_filtrado = "07317044000199"  # Exemplo do CNPJ completamente numérico
df_cnpj = df_anp_adv[df_anp_adv["CNPJ"] == cnpj_filtrado]

In [ ]:
df_cnpj = df_cnpj.sort_values(by="DtColeta")

cnpj = df_cnpj["CNPJ"].iloc[0]
nome_revenda = df_cnpj["Revenda"].iloc[0]

# Filter periods where 'ajuste_preco' equals 1
adjustment_periods = df_cnpj[df_cnpj["ajuste_preco"] == 1]["DtColeta"]

# Criar uma figura de linha para o preço VENDA
fig = go.Figure()

# Adicionar a linha de preços VENDA ao longo do tempo
fig.add_trace(
    go.Scatter(
        x=df_cnpj["DtColeta"],  # Eixo X como DtColeta, que está em formato datetime
        y=df_cnpj["Venda"],  # Eixo Y como preço de venda
        mode="lines",
        name="Preço VENDA",
    )
)

# Iterate through adjustment periods and add shaded regions
for start_date in adjustment_periods:
    # Assuming each period lasts 14 days as defined by your previous setup
    end_date = start_date + pd.Timedelta(days=13)
    fig.add_vrect(
        x0=start_date,
        x1=end_date,
        fillcolor="red",
        opacity=0.2,
        layer="below",
        line_width=0,
    )

# Definir títulos e ajustes nos eixos
fig.update_layout(
    title=f"Comportamento do Preço de Venda da Gasolina - CNPJ: {cnpj}, Revenda: {nome_revenda}",
    xaxis_title="Data de Coleta",
    yaxis_title="Preço de Venda (R$)",
    xaxis=dict(type="date"),  # Forçar o eixo X a reconhecer como datas
    showlegend=True,
)

# Mostrar o gráfico
fig.show()

### Stats: Algumas estatisticas descritivas

#### Total de revendass por Ano e Bandeira

In [ ]:
stats_1 = (
    df_anp_adv.groupby(["Ano", "Bandeira"], as_index=True, sort=True)
    .agg(
        {
            "NomRevenda": ["nunique"],
        }
    )
    .round(2)
)

stats_1.columns = stats_1.columns.droplevel(1)
stats_1.rename(columns={"NomRevenda": "#_revendas"}, inplace=True)
stats_1.reset_index(inplace=True)
stats_1.sort_values(by=["Ano", "Bandeira"], inplace=True)
stats_1

#### Plot: Total de revenda por Esado e Bandeira

In [ ]:
stats_2 = (
    df_anp_adv.groupby(["UF", "Bandeira"], as_index=True, sort=True)
    .agg(
        {
            "NomRevenda": ["nunique"],
        }
    )
    .round(2)
)

stats_2.columns = stats_2.columns.droplevel(1)
stats_2.reset_index(inplace=True)


## Análise gráfica
stats_2.sort_values(by=["UF", "Bandeira"], ascending=[True, True], inplace=True)
plt.figure(figsize=(12, 5))
sns.barplot(data=stats_2, x="UF", y="NomRevenda", hue="Bandeira")

plt.title("Total Revendas por Bandeira por UF")
plt.xlabel("")
plt.ylabel("Total de Revendas")
plt.xticks(rotation=45)
plt.legend(title="Bandeira")
plt.tight_layout()

plt.show()

### Estatisticas de Mercado

Recriar as metricas de mercado 

In [ ]:
stats_1 = (
    df_anp_adv.groupby(
        [
            "DtColeta",
            "Ano",
            "Mes",
            "Regiao",
            "UF",
            "Bandeira",
            "NomMunicipio",
        ],
        as_index=True,
        sort=True,
    )
    .agg(
        {
            "NomRevenda": ["nunique"],
            "Venda": ["mean", "std"],
        }
    )
    .round(2)
)

stats_1.columns = stats_1.columns.droplevel(1)
stats_1.rename(columns={"NomRevenda": "#_revendas"}, inplace=True)
stats_1.reset_index(inplace=True)
# stats_1.sort_values(by=["DtColeta", "Bandeira"], inplace=True)

stats_1.columns = [
    "DtColeta",
    "Ano",
    "Mes",
    "Regiao",
    "UF",
    "Bandeira",
    "Municipio",
    "#_Revendas",
    "Preco_venda_medio",
    "Preco_venda_std",
]

stats_1

### Verificando se existe bandeira VIBRA no municipio

In [ ]:
# Step 1: Identify rows where Bandeira is VIBRA
stats_1["VIBRA_in_Municipio"] = stats_1["Bandeira"] == "VIBRA"

# Step 2: Group by Municipio and evaluate if VIBRA exists in that Municipio
stats_1["VIBRA_in_Municipio"] = stats_1.groupby("Municipio")[
    "VIBRA_in_Municipio"
].transform("max")

# Step 3: Convert the boolean flag to an integer (0 or 1)
stats_1["VIBRA_in_Municipio"] = stats_1["VIBRA_in_Municipio"].astype(int)

display(stats_1)

# Filter and print municipalities that don't have VIBRA
municipios_without_vibra = stats_1[stats_1["VIBRA_in_Municipio"] == 0][
    ["Municipio", "VIBRA_in_Municipio"]
].drop_duplicates()

print(municipios_without_vibra.head(10))

### Manter apenas municipios que tem VIBRA

In [ ]:
stats_1 = stats_1[stats_1["VIBRA_in_Municipio"] == 1]

### Verificando se houve coleta de informação em VIBRA no mês na cidade

In [ ]:
# Step 1: Identify rows where Bandeira is VIBRA
stats_1["VIBRA_Mes"] = stats_1["Bandeira"] == "VIBRA"

# Step 2: Group by Year, Month, and City to create a Mes for each group
stats_1["VIBRA_Mes"] = stats_1.groupby(["Ano", "Mes", "Municipio"])[
    "VIBRA_Mes"
].transform("max")

# Step 3: Convert the boolean Mes to an integer (0 or 1)
stats_1["VIBRA_Mes"] = stats_1["VIBRA_Mes"].astype(int)

In [ ]:
# Pivot the DataFrame
df_vibra = stats_1.pivot_table(
    index=["DtColeta", "Ano", "Mes", "Dia", "Regiao", "UF", "Municipio"],
    columns="Bandeira",
    values="Preco_venda_medio",
    aggfunc="mean",  # or "max", "min", depending on what you want
)

# Reset the index to flatten the DataFrame
df_vibra.reset_index(inplace=True)

df_vibra.sort_values(by=["DtColeta", "Municipio"], ascending=True)